-----

## Batcher Sorting Networks: Merge Sort

Burton Rosenberg

_Creation Date:_ June 2023

_Last update:_ 26 September 2024

&copy; Copyright 2023 Burton Rosenberg. All rights reserved.


----


### Table of contents.

1. <a href="#introduction">Sorting Networks</a>
1. <a href="#01principle">0-1 Principle</a>
1. <a href="#oddeven">Batcher Odd-Even Sorting Network</a>
1. <a href="#serial">Serial Implementation</a>


### <a name=introduction>Sorting Networks</a>

The question is how to sort in parallel, especially if the parallelism can be exploited for a faster sort.

It is possible to realize parallel compution with a circuit consisting of computing units and wires connectin the units, or on a device such as a GPU that has a common memory accessible to multiple computing threads. In sorting networks, the computation consists of layers or either permutation of wires or a swap gate connecting two neighboring wires. The swap gate implements 

$$\mathcal{S}(x,y):=(\min(x,y),\max(x,y)$$ 

These circuits for sorting are called _sorting networks_. 

Here is a sorting network for four elements, based on the Bacher Odd-Even algorithm. The wires are the horizontal lines and slanted lines, and the computation units are the vertical lines. Verify that this circuit works.

<pre>
EXAMPLE 1

A ---+----------+------- A
     |          |
B ---+--+    +--+--+---  a
          \ /      |
           /       |
          / \      |
a ---+---+   +--+--+---  B
     |          |
b ---+----------+------- b
</pre>

### <a name=01principle>0-1 Principle</a>

If a sorting network works when the values to sort are restricted to $\{\,0,1\,\}$, then it works in general. The proof of this can be found in Corman, Leiserson, Rivest and Stein. Also in that book is a great discussion off sorting networks and the Batcher sorts we consider here.



### <a name=oddeven>Batcher Odd-Even Merge Network</a>


In a 1968 report, Ken Batcher presented two sorting networks that have $O((\log n)^2)$ layers. Since each layer is computed in unit time, either as a circuit or on a GPU, the time to sort is also $O((\log n)^2)$. In the circuit model $O(n\,(\log n)^2)$ swap units are needed. In the GPU model, $O(n)$ threads are needed in each thead launch.


The first is the odd-even merge network. We will describe the algorithm for general integers but prove it using the 0--1 principle. 



The algorithm is based on a Merge device $M_n$. A merge device takes two $n$ bit inputs, $a$ and $b$ each sorted, and gives a $2n$ sorted output $c$. Denoting by $\#a$ then number of ones in $a$, likewise $b$ and $c$, the _correctness_ condition is $\#c=\#a+\#b$.

<pre>

M_1

 A----+----A
      |
 a----+----a
  
M_2

  A---------+-------A
            | 
  B---+  +--+---+---a
       \/       |
       /\       |
  a---+  +--+---+---B
            |
  b---------+-------b

</pre>

__Examples:__ $M_1$ is a swap. For $M_2$, there are 9 possible inputs to consider, since the two upper and lower pairs are allowed to be. 00, 01, and 11. (The sort will result in ascending values.)





The heart of the algorithm is the division of $a$ and $b$ each into two sequence by extracting in order the even indexed bits, $a_e$ and $b_e$, and the odd indexed bits $a_o$ and $b_o$. (Bits are counted starting with index zero). Two calls to $M_{n/2}$, recusively defined, give,

\begin{eqnarray*}
c_e &=& M_{n/2}(a_e,b_e) \\
c_o &=& M_{n/2}(a_o,b_o)
\end{eqnarray*}

The output $c$ is the merge of $c_e$ and $c_o$, where

- the top bit of $c_e$ becomes the top bit of $c$, 
- the bottom bit of $c_o$ becomes the bottom bit of $c$
- the remaining $n-2$ bits are paired, alternating a bit from $c_e$ with a bit from $c_o$, and the pair sent through a swap gate.

And essential element of the algorithm is that $0\le\#c_e-\#c_o\le 2$.

- If $\#a$ and $\#b$ are both even, they split their ones evenly into the even-odd lists, and the same number of ones go into each of the recursive merges.
- If $\#a$ and $\#b$ are both odd, the even lists each receive one more one than the odd lists, and when combined as input ot the recursive merge, therefore there are two more ones on the merge of even lists than odd lists.
- In the remaining case there is one more one in the merge of the even lists than the odd lists.

The combining layer forwards to the output the top wire of $c_e$. If this wire is a zero, the output is all zeros and we are done. The bottom wire of $c_o$ is also fowarded. Calling this gleened lists with a prime,
$-1\le\#c'_e-\#c'_o\le 1$.

The lists are paired elementwise. Depending on if $c'_e$ has fewer, same, or more ones than $c'_o$, there will
be a pairing $01$, $11$ or $10$, respectively. All other pairings are $00$ or $11$. A final swap is applied to these pairs so that if the pairing is $01$, it becomes $10$. The sort is complete.


__Recursive construction:__

By the recursive construction, 

$$\text{Depth}(M_n) = \Theta(\log n)$$

The merger device $M_n$ results in a sort list of $n$ wires, given two sorted list of $n/2$. The top and 
bottom half of the given data is sorted by two $M_{n/2}$ devices to achieve this; and interatively four
$M_{n/4}$ devices supply the proper input to the two $M_{n/2}$ devices. And so on to $\Theta(\log n)$ levels.


Giving an overall circuit depth of $\Theta(\log^2 n)$.


In [1]:
class SortingNetwork:
    
    def __init__(self,n,phases):
        self.phases = phases
        self.n = n
        
    def run(self,c_in,is_verbose=False):
        assert len(c_in)==self.n, 'not the correct circuit size'
        
        c = c_in[:]
        if is_verbose:
            print(f'I:\t{c}')
        for phase in self.phases:
            if phase[0]=='P':
                c = self.permute(phase[1],c)
            if phase[0]=='S':
                c = self.swap(phase[1],c)
            if is_verbose:
                print(f'{phase[0]}:\t{c}')
        return c

    def permute(self,inst,c):
        c_out = [0]*len(c)
        for i in range(self.n):
            c_out[i] = c[inst[i]]
        return c_out
    
    def swap(self,inst,c):
        for i in inst:
            c[i],c[i+1] = min(c[i],c[i+1]), max(c[i],c[i+1])
        return c


In [2]:
import random

example_1 = [('S',[0,2]),('P',[0,2,1,3]),('S',[0,2]),('S',[1])]
sn = SortingNetwork(4,example_1)
for trials in range(4):
    c = [i for i in range(4)]
    random.shuffle(c)
    print(f'in: {c}\nout:{sn.run(c,is_verbose=True)}\n')
    
merge_by4 = [
    ('P',[0,2,4,6,1,3,5,7]),
    ('P',[0,2,1,3,4,6,5,7]),
    ('S',[0,2,4,6]),
    ('S',[1,5]),
    ('P',[0,1,4,2,5,3,6,7]),
    ('S',[1,3,5])
     ]

sn = SortingNetwork(8,merge_by4)

for trials in range(4):
    c = [(i%4) for i in range(8)]
    for i in range(4):
        c[i] = (trials%2+2)*c[i]
        c[i+4] = (trials%3+1)*c[i+4]
    print(f'in: {c}\nout:{sn.run(c,is_verbose=True)}\n')


I:	[1, 3, 0, 2]
S:	[1, 3, 0, 2]
P:	[1, 0, 3, 2]
S:	[0, 1, 2, 3]
S:	[0, 1, 2, 3]
in: [1, 3, 0, 2]
out:[0, 1, 2, 3]

I:	[1, 3, 0, 2]
S:	[1, 3, 0, 2]
P:	[1, 0, 3, 2]
S:	[0, 1, 2, 3]
S:	[0, 1, 2, 3]
in: [1, 3, 0, 2]
out:[0, 1, 2, 3]

I:	[3, 2, 1, 0]
S:	[2, 3, 0, 1]
P:	[2, 0, 3, 1]
S:	[0, 2, 1, 3]
S:	[0, 1, 2, 3]
in: [3, 2, 1, 0]
out:[0, 1, 2, 3]

I:	[2, 0, 1, 3]
S:	[0, 2, 1, 3]
P:	[0, 1, 2, 3]
S:	[0, 1, 2, 3]
S:	[0, 1, 2, 3]
in: [2, 0, 1, 3]
out:[0, 1, 2, 3]

I:	[0, 2, 4, 6, 0, 1, 2, 3]
P:	[0, 4, 0, 2, 2, 6, 1, 3]
P:	[0, 0, 4, 2, 2, 1, 6, 3]
S:	[0, 0, 2, 4, 1, 2, 3, 6]
S:	[0, 0, 2, 4, 1, 2, 3, 6]
P:	[0, 0, 1, 2, 2, 4, 3, 6]
S:	[0, 0, 1, 2, 2, 3, 4, 6]
in: [0, 2, 4, 6, 0, 1, 2, 3]
out:[0, 0, 1, 2, 2, 3, 4, 6]

I:	[0, 3, 6, 9, 0, 2, 4, 6]
P:	[0, 6, 0, 4, 3, 9, 2, 6]
P:	[0, 0, 6, 4, 3, 2, 9, 6]
S:	[0, 0, 4, 6, 2, 3, 6, 9]
S:	[0, 0, 4, 6, 2, 3, 6, 9]
P:	[0, 0, 2, 4, 3, 6, 6, 9]
S:	[0, 0, 2, 3, 4, 6, 6, 9]
in: [0, 3, 6, 9, 0, 2, 4, 6]
out:[0, 0, 2, 3, 4, 6, 6, 9]

I:	[0, 2, 4, 6, 0, 3, 6, 9]
P:


### <a name=serial>Serial Implementation</a>

Follows it python code carrying out the sort; however this code does not attempt to simulate a network.


In [3]:
# batcher's even odd sort

class BatcherOddEvenMerge:
    
    def __init__(self):
        pass
    
    def merge_aux(self,a,b):
        
        if len(a)==1:
            return [min(a[0],b[0]),max(a[0],b[0])]

        # using batcher's numbering for odd and even 
        # (contrary to based at zero arrays)
        a_odd = a[0::2]
        a_even = a[1::2]
        b_odd = b[0::2]
        b_even = b[1::2]

        odd = self.merge_aux(a_odd,b_odd)
        even = self.merge_aux(a_even,b_even)

        c = [0]*(len(a)+len(b))
        c[0] = odd[0]
        c[-1] = even[-1]
        c[1:-1:2] = odd[1::]
        c[2:-1:2] = even[0:len(even)-1]
        for i in range(1,len(c)-1,2):
            if c[i]>c[i+1] : c[i],c[i+1]=c[i+1],c[i]
        return c
    
    @staticmethod
    def power_of_two(n):
        while n>1:
            if n%2==1:
                return False
            n //= 2
        return True

    def merge(self,a,b):
        
        assert len(a)==len(b), 'lists must be of equal size'
        assert BatcherOddEvenMerge.power_of_two(len(a)), 'list length must be a power of 2'
        return self.merge_aux(a,b)
    
    def sort(self,c):
        BatcherOddEvenMerge.power_of_two(len(c))
        q = []
        for c_ele in c:
            q.append([c_ele])
        while len(q)>1:
            a = q.pop(0)
            b = q.pop(0)
            q.append(self.merge(a,b))
        return q[0]


In [4]:
import random

k= 8
a = [3*i for i in range(k)]

print('a:',a)
b = [2*i for i in range(k)]

print('b:',b)
bod = BatcherOddEvenMerge()
print('merged:\n',bod.merge(a,b))

k= 16
c = [i for i in range(k)]
random.shuffle(c)
print('c:',c)
bod = BatcherOddEvenMerge()
print('sort:\n',bod.sort(c))


a: [0, 3, 6, 9, 12, 15, 18, 21]
b: [0, 2, 4, 6, 8, 10, 12, 14]
merged:
 [0, 0, 2, 3, 4, 6, 6, 8, 9, 10, 12, 12, 14, 15, 18, 21]
c: [2, 5, 15, 4, 0, 12, 7, 1, 10, 11, 3, 6, 9, 14, 8, 13]
sort:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


### END